In [ ]:
# Examples of Using Big Query as backend to Query data.
# Examples assume 
# a. google auth login - commands have been run to set up auth and default project
# b. the default project has `synthea_big` dataset with Patient, Encounter and Observation tables with sample data
# that is used in this notebook

In [18]:
import warnings
from query_lib_big_query import _BigQueryPatientQuery

_BIGQUERY_DATASET = "synthea_big"
_CODE_SYSTEM = "http://www.ampathkenya.org"
warnings.filterwarnings("ignore")

# Simple Example to Fetch Encounter data

In [23]:
pq = _BigQueryPatientQuery(
        bq_dataset=_BIGQUERY_DATASET, code_system="dummy_code_system"
    )
pq.encounter_constraints(
        typeSystem="http://fhir.openmrs.org/code-system/encounter-type"
    )

encounter_df = pq.get_patient_encounter_view(
        base_url="", force_location_type_columns=False
    )
encounter_df.head(5)

,encPatientId,locationId,encTypeSystem,encTypeCode,locationDisplay,num_encounters,firstDate,lastDate
0,280cc47f-d08f-4bc1-bd1b-4378bcbd1f86,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,1,2021-06-24T02:58:07+00:00,None
1,76adaa5a-170a-4302-a377-0a325ec1e69e,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,1,1967-11-26T04:12:27+00:00,None
2,d6ff8f55-acc7-4eee-b5af-acb7a878add8,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,1,2005-11-09T08:26:04+00:00,None
3,76adaa5a-170a-4302-a377-0a325ec1e69e,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,1,1969-11-23T04:12:27+00:00,None
4,195bac00-85e3-4095-b2df-ffe2d79e242a,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,1,1990-04-25T19:26:21+00:00,None


# Fetch Encounter Data based on codes

In [24]:
pq = _BigQueryPatientQuery(
        bq_dataset=_BIGQUERY_DATASET, code_system="dummy_code_system"
    )
test_codes = [
        "e22e39fd-7db2-45e7-80f1-60fa0d5a4378",
        "181820aa-88c9-479b-9077-af92f5364329",
    ]

pq.encounter_constraints(typeCode=test_codes)

encounter_df = pq.get_patient_encounter_view(
        base_url="", force_location_type_columns=False
    )
encounter_df.head(5)

,encPatientId,locationId,encTypeSystem,encTypeCode,locationDisplay,num_encounters,firstDate,lastDate
0,8295eb5b-fba6-4e83-a5cb-2817b135cd27,b1a8b05e-3542-4037-bbd3-998ee9c40574,http://fhir.openmrs.org/code-system/encounter-...,e22e39fd-7db2-45e7-80f1-60fa0d5a4378,Inpatient Ward,1,2019-02-28T10:43:00+00:00,None
1,8295eb5b-fba6-4e83-a5cb-2817b135cd27,b1a8b05e-3542-4037-bbd3-998ee9c40574,http://fhir.openmrs.org/code-system/encounter-...,181820aa-88c9-479b-9077-af92f5364329,Inpatient Ward,1,2019-03-03T10:43:00+00:00,None
2,8295eb5b-fba6-4e83-a5cb-2817b135cd27,b1a8b05e-3542-4037-bbd3-998ee9c40574,http://fhir.openmrs.org/code-system/encounter-...,e22e39fd-7db2-45e7-80f1-60fa0d5a4378,Inpatient Ward,1,2020-12-07T10:58:01+00:00,None
3,8295eb5b-fba6-4e83-a5cb-2817b135cd27,b1a8b05e-3542-4037-bbd3-998ee9c40574,http://fhir.openmrs.org/code-system/encounter-...,181820aa-88c9-479b-9077-af92f5364329,Inpatient Ward,1,2020-12-08T10:58:01+00:00,None


# Fetch Encounter data based on location id

In [25]:
pq = _BigQueryPatientQuery(
        bq_dataset=_BIGQUERY_DATASET, code_system="dummy_code_system"
    )

test_locations = ["2131aff8-2e2a-480a-b7ab-4ac53250262b"]

pq.encounter_constraints(locationId=test_locations)

encounter_df = pq.get_patient_encounter_view(
        base_url="", force_location_type_columns=False
    )
encounter_df.head(5)

,encPatientId,locationId,encTypeSystem,encTypeCode,locationDisplay,num_encounters,firstDate,lastDate
0,280cc47f-d08f-4bc1-bd1b-4378bcbd1f86,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,1,2021-06-24T02:58:07+00:00,None
1,76adaa5a-170a-4302-a377-0a325ec1e69e,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,1,1967-11-26T04:12:27+00:00,None
2,d6ff8f55-acc7-4eee-b5af-acb7a878add8,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,1,2005-11-09T08:26:04+00:00,None
3,76adaa5a-170a-4302-a377-0a325ec1e69e,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,1,1969-11-23T04:12:27+00:00,None
4,195bac00-85e3-4095-b2df-ffe2d79e242a,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,1,1990-04-25T19:26:21+00:00,None


# Encounter data with multiple condition

In [ ]:
test_locations = ["b1a8b05e-3542-4037-bbd3-998ee9c40574"]
test_codes = [
    "e22e39fd-7db2-45e7-80f1-60fa0d5a4378",
    "181820aa-88c9-479b-9077-af92f5364329",
]
test_type_system = "http://fhir.openmrs.org/code-system/encounter-type"

pq = _BigQueryPatientQuery(
    bq_dataset=_BIGQUERY_DATASET, code_system="dummy_code_system"
)
pq.encounter_constraints(
    typeSystem=test_type_system,
    locationId=test_locations,
    typeCode=test_codes
)
encounter_df = pq.get_patient_encounter_view(
    base_url="",
    force_location_type_columns=False,
    sample_count=10,
)
encounter_df.head(5)

# Observation Data with multiple conditions

In [27]:
_VL_CODE = "856"  # HIV VIRAL LOAD
_ARV_PLAN = "1255"  # ANTIRETROVIRAL PLAN
end_date = "2017-01-01"
start_date = "2000-01-01"
_BASE_URL = ""

# Creating a new `patient_query` to drop all previous constraints
# and recreate flat views.
patient_query = _BigQueryPatientQuery(
    bq_dataset=_BIGQUERY_DATASET, code_system=_CODE_SYSTEM
)

patient_query.include_obs_values_in_time_range(
    _VL_CODE, min_time=start_date, max_time=end_date
)
patient_query.include_obs_values_in_time_range(
    _ARV_PLAN, min_time=start_date, max_time=end_date
)

other_end_date = "2020-01-01"
other_start_date = "1998-01-01"
patient_query.include_all_other_codes(
    min_time=other_start_date, max_time=other_end_date
)

patient_query.encounter_constraints(
    locationId=["2131aff8-2e2a-480a-b7ab-4ac53250262b"]
)

obs_df = patient_query.get_patient_obs_view(_BASE_URL)
obs_df.head(5)

      max_date_value  ...             min_date_value_code
0               None  ...  2009-11-13T08:36:26+00:00,5424
1               None  ...  2001-10-30T21:50:18+00:00,6180
2               None  ...  2010-09-06T23:26:35+00:00,6180
3               None  ...  2012-07-26T21:29:27+00:00,5424
4               None  ...  2000-06-01T14:36:24+00:00,5424
...              ...  ...                             ...
24509           None  ...  2015-08-24T17:09:45+00:00,1111
24510           None  ...  2015-02-23T05:51:23+00:00,1032
24511           None  ...  2006-05-07T21:41:45+00:00,1107
24512           None  ...  2009-10-10T00:19:40+00:00,1107
24513           None  ...  2007-03-22T16:49:47+00:00,1107

[24514 rows x 4 columns]


,patientId,code,birthDate,gender,num_obs,min_value,max_value,min_date,max_date,last_value,first_value,last_value_code,first_value_code
0,4b5405d2-1b67-4e9f-af52-6e75712fdc37,1088,1990-12-23,female,33,NaN,NaN,2009-11-13T08:36:26+00:00,2016-04-20T08:36:26+00:00,None,None,815,5424
1,2556c875-bbd9-4f22-848e-be311eed676f,1088,1979-07-11,female,29,NaN,NaN,2001-10-30T21:50:18+00:00,2004-10-09T21:50:18+00:00,None,None,815,6180
2,c16075d3-f237-4d00-9c43-030fe76c2fff,1088,1991-05-21,female,32,NaN,NaN,2010-09-06T23:26:35+00:00,2013-06-11T23:26:35+00:00,None,None,815,6180
3,e623d619-0208-4958-8f65-a62cc28eab01,1088,1988-11-27,female,33,NaN,NaN,2012-07-26T21:29:27+00:00,2017-11-17T21:29:27+00:00,None,None,806,5424
4,1cb2ba13-cace-4aa3-9159-11ff531e0ae3,1088,1974-05-22,female,59,NaN,NaN,2000-06-01T14:36:24+00:00,2006-10-23T14:36:24+00:00,None,None,806,5424
